In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端机器学习： MLOps阶段1：数据管理

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/mlops_data_management.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage1/mlops_data_management.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

*注意：此笔记本不支持在Colab中执行*

## 概述

本教程演示如何在Google Cloud上生产环境中使用Vertex AI进行端到端MLOps。本教程涵盖阶段1：数据管理。

### 目标

在本教程中，您将创建一个 MLOps 阶段 1：数据管理流程。

本教程使用以下 Vertex AI 和数据分析服务：

- `Vertex AI 数据集`
- `BigQuery`
- `Dataflow`

执行的步骤包括：

- 探索和可视化数据。
- 从`BigQuery`表创建一个 Vertex AI `数据集` 资源 -- 用于 AutoML 训练。
- 将数据集的副本提取到 Cloud 存储中的 CSV 文件。
- 从 CSV 文件创建一个 Vertex AI `数据集` 资源 -- 作为 AutoML 训练的另一种选择。
- 将`BigQuery`数据集的样本读取到 dataframe 中。
- 使用 TensorFlow 数据验证从 dataframe 中的样本生成统计数据和数据架构。
- 使用 TensorFlow 数据验证从数据架构生成 TFRecord 特征规范。
- 使用`Dataflow`预处理部分`BigQuery`数据 -- 用于自定义训练。

### 推荐

在谷歌云上进行端到端MLOps数据管理时，建议使用以下结构化（表格）数据的最佳实践：

- 对于大量数据，请使用BigQuery表。否则，请使用存储在云存储中的CSV文件。
- 在CSV文件中存储大量数据时，请将数据分片为每个分片的10,000行。
- 使用Vertex AI的`TabularDataset`创建受管数据集。
- 使用`Dataflow`对数据进行预处理。

### 数据集

本教程使用的数据集是[芝加哥出租车](https://www.kaggle.com/chicago/chicago-taxi-trips-bq)。本教程中使用的数据集版本存储在一个公共的BigQuery表中。经过训练的模型预测某人是否会为出租车服务留下小费。

成本
本教程使用了Google Cloud的计费组件：

- Vertex AI
- Cloud Storage
- BigQuery
- Dataflow

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)，[Cloud Storage定价](https://cloud.google.com/storage/pricing)，[BigQuery定价](https://cloud.google.com/bigquery/pricing)，以及[Dataflow定价](https://cloud.google.com/dataflow/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您预计的使用情况生成成本估算。

安装

仅需安装一次用于执行 MLOps 笔记本的软件包。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

ONCE_ONLY = False
if ONCE_ONLY:
    ! pip3 install -U {USER_FLAG} -q tensorflow==2.5 \
                                     tensorflow-data-validation==1.2 \
                                     tensorflow-transform==1.2 \
                                     tensorflow-io==0.18 
    
    ! pip3 install --upgrade {USER_FLAG} -q google-cloud-aiplatform[tensorboard] \
                                            google-cloud-pipeline-components \
                                            google-cloud-bigquery \
                                            google-cloud-logging \
                                            apache-beam[gcp] \
                                            pyarrow \
                                            cloudml-hypertune


重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

开始之前

设置您的Google Cloud项目

**无论您使用哪种笔记本环境，下面的步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用额度用于计算/存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI、BigQuery、计算引擎和云存储API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,bigquery,compute_component,storage_component)。

1. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**: Jupyter会将以`!`开头的行视为shell命令，并将以`$`开头的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于笔记本的其余部分的操作。以下是Vertex AI支持的区域。我们建议您选择最接近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法在Vertex AI进行训练时使用多区域存储桶。并非所有区域都支持所有Vertex AI服务。

了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在进行实时教程会话，您可能正在使用共享的测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud账户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经通过验证。

**如果您正在使用Colab**，运行下方的单元格，并按照提示进行身份验证，通过oAuth授权您的账户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，前往[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

1. **点击创建服务帐户**。

2. 在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

3. 在**将此服务帐户授权访问项目**部分，点击角色下拉列表。在过滤框中输入"Vertex AI"，选择**Vertex AI管理员**。在过滤框中输入"Storage Object Admin"，选择**Storage Object Admin**。

4. 点击创建。包含您密钥的JSON文件将下载到您的本地环境。

5. 在下方的单元格中，将您的服务帐户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用什么笔记本环境，都需要执行以下步骤。**

当您使用 Vertex AI SDK 提交一个自定义训练任务时，您需要上传一个包含训练代码的 Python 软件包到一个云存储桶中。Vertex AI 将从该软件包中运行代码。在本教程中，Vertex AI 还会将您的任务的训练模型保存在同一个存储桶中。您可以随后基于这个输出创建一个 `Endpoint` 资源，以便用于提供在线预测。

在下面设置您的云存储桶的名称。存储桶的名称在所有 Google Cloud 项目中必须是全局唯一的，包括您所在组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过查看其内容来验证对Cloud Storage存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在整个教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

导入BigQuery

将BigQuery包导入到您的Python环境中。

In [ ]:
from google.cloud import bigquery

#### 导入Apache Beam

将Apache Beam包导入到您的Python环境中。

In [ ]:
import apache_beam as beam

导入TensorFlow

将TensorFlow包导入您的Python环境中。

In [ ]:
import tensorflow as tf

导入TensorFlow数据验证（TFDV）包到您的Python环境中。

In [ ]:
import tensorflow_data_validation as tfdv

#### 导入TensorFlow Transform

将TensorFlow Transform（TFT）包导入到您的Python环境中。

In [ ]:
import tensorflow_transform as tft

### 初始化Python的Vertex AI SDK

为您的项目和对应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

创建BigQuery客户端。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

数据集

接下来，您将看看创建受管数据集的选项：

* `BigQuery`：创建一个 Vertex `TabularDataset` 资源。
* `CSV`：创建一个 Vertex `TabularDataset` 资源。
* `TFRecords`：在云存储上自行管理数据集。

#### BigQuery培训数据的位置。

现在将变量`IMPORT_FILE`设置为BigQuery中数据表的位置。

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.chicago_taxi_trips.taxi_trips"
BQ_TABLE = "bigquery-public-data.chicago_taxi_trips.taxi_trips"

### 探索 BigQuery 数据集

探索 BigQuery 表格的内容：

- 获取所有来自 2015 年的示例
- 按照一周中的日期排序
- 计算每周各天的示例数量.

In [ ]:
query = """SELECT
    CAST(EXTRACT(DAYOFWEEK FROM trip_start_timestamp) AS string) AS trip_dayofweek,
    FORMAT_DATE('%A',cast(trip_start_timestamp as date)) AS trip_dayname,
    COUNT(*) as trip_count,
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    EXTRACT(YEAR FROM trip_start_timestamp) = 2015
GROUP BY
    trip_dayofweek,
    trip_dayname
ORDER BY
    trip_dayofweek"""

_ = bqclient.query(query)
rows = _.result()
dataframe = rows.to_dataframe()
print(dataframe.head(7))

In [ ]:
dataframe.plot(kind="bar", x="trip_dayname", y="trip_count")

### 创建BigQuery表的私有副本

接下来，您可以创建BigQuery表的私有副本：
- 选择列的子集
- 选择行的子集（限制）
- 设置条件（WHERE）
- 对地理位置坐标进行特征工程
- 预先拆分数据集

In [ ]:
BQ_DATASET = BQ_TABLE.split(".")[1]
BQ_TABLE_COPY = f"{PROJECT_ID}.{BQ_DATASET}.taxi_trips"
LIMIT = 300000
YEAR = 2020

# First, create the dataset entry
dataset = bigquery.Dataset(f"{PROJECT_ID}.{BQ_DATASET}")
dataset.location = "US"
dataset = bqclient.create_dataset(dataset, timeout=30)

query = f"""
CREATE OR REPLACE TABLE `{BQ_TABLE_COPY}`
AS (
    WITH
      taxitrips AS (
      SELECT
        trip_start_timestamp,
        trip_seconds,
        trip_miles,
        payment_type,
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude,
        tips,
        fare
      FROM
        `{BQ_TABLE}`
      WHERE pickup_longitude IS NOT NULL
      AND pickup_latitude IS NOT NULL
      AND dropoff_longitude IS NOT NULL
      AND dropoff_latitude IS NOT NULL
      AND trip_miles > 0
      AND trip_seconds > 0
      AND fare > 0
      AND EXTRACT(YEAR FROM trip_start_timestamp) = {YEAR}
    )

    SELECT
      EXTRACT(MONTH from trip_start_timestamp) as trip_month,
      EXTRACT(DAY from trip_start_timestamp) as trip_day,
      EXTRACT(DAYOFWEEK from trip_start_timestamp) as trip_day_of_week,
      EXTRACT(HOUR from trip_start_timestamp) as trip_hour,
      CAST(trip_seconds AS FLOAT64) as trip_seconds,
      trip_miles,
      payment_type,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(pickup_longitude, pickup_latitude), 0.1)
      ) AS pickup_grid,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(dropoff_longitude, dropoff_latitude), 0.1)
      ) AS dropoff_grid,
      ST_Distance(
          ST_GeogPoint(pickup_longitude, pickup_latitude),
          ST_GeogPoint(dropoff_longitude, dropoff_latitude)
      ) AS euclidean,
      CONCAT(
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickup_longitude,
              pickup_latitude), 0.1)),
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropoff_longitude,
              dropoff_latitude), 0.1))
      ) AS loc_cross,
      IF((tips/fare >= 0.2), 1, 0) AS tip_bin,
    FROM
      taxitrips
    LIMIT {LIMIT}
)
"""

response = bqclient.query(query)
_ = response.result()

BQ_TABLE = BQ_TABLE_COPY
IMPORT_FILE = f"bq://{BQ_TABLE_COPY}"

### 创建数据集

#### BigQuery 输入数据

接下来，使用 `TabularDataset` 类的 `create` 方法创建 `Dataset` 资源，该方法需要以下参数：

- `display_name`：`Dataset` 资源的人类可读名称。
- `bq_source`：将数据项从 BigQuery 表导入到 `Dataset` 资源中。
- `labels`：用户定义的元数据。在此示例中，您可以存储包含用户定义数据的 Cloud Storage 存储桶的位置。

了解更多关于 [TabularDataset from BigQuery table](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_bigquery_sample-python)。

In [ ]:
dataset = aip.TabularDataset.create(
    display_name="Chicago Taxi" + "_" + TIMESTAMP,
    bq_source=[IMPORT_FILE],
    labels={"user_metadata": BUCKET_NAME},
)

label_column = "tip_bin"

print(dataset.resource_name)

### 读取 BigQuery 数据集到 pandas dataframe

接下来，您可以使用 BigQuery `list_rows()` 和 `to_dataframe()` 方法，将数据集的样本读取到 pandas dataframe 中，具体步骤如下：

- `list_rows()`: 对指定表执行查询并返回查询结果的行迭代器。可选指定：
  - `selected_fields`: 要返回的字段（列）的子集。
  - `max_results`: 要返回的最大行数。与 SQL LIMIT 命令相同。

- `rows.to_dataframe()`: 调用行迭代器并将数据读取到 pandas dataframe 中。

了解更多关于[将 BigQuery 表加载到 dataframe 中的内容](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas)。

In [ ]:
# Download a table.
table = bigquery.TableReference.from_string(BQ_TABLE)

rows = bqclient.list_rows(table, max_results=300000)

dataframe = rows.to_dataframe()
print(dataframe.head())

### 生成数据集统计

#### 数据框输入数据

使用 TensorFlow Data Validation（TFDV）包对数据集生成统计数据。使用`generate_statistics_from_dataframe()`方法，并设置以下参数：

- `dataframe`：存储在内存中的pandas数据框中的数据集。
- `stats_options`：已选统计选项：
  - `label_feature`：要预测的列。
  - `sample_rate`：抽样率。如果指定，则对样本进行统计。
  - `num_top_values`：要保留的最常见特征值数量。

了解有关[TensorFlow Data Validation（TFDV）](https://www.tensorflow.org/tfx/data_validation/get_started)的信息。

In [ ]:
stats = tfdv.generate_statistics_from_dataframe(
    dataframe=dataframe,
    stats_options=tfdv.StatsOptions(
        label_feature="tip_bin", sample_rate=1, num_top_values=50
    ),
)

print(stats)

### 可视化数据集统计信息

可以使用TFDV的`visualize_statistics()`方法来显示数据集统计信息的可视化。

In [ ]:
tfdv.visualize_statistics(stats)

### 从统计数据中提取特征分组

接下来，您可以从统计数据中提取特征名称和数据类型，然后将这些特征分为：

- 数值特征：浮点数
- 分类特征：字符串、整数

In [ ]:
NUMERIC_FEATURES = []
CATEGORICAL_FEATURES = []
for _ in range(len(stats.datasets[0].features)):
    if stats.datasets[0].features[_].path.step[0] == label_column:
        continue
    if stats.datasets[0].features[_].type == 0:  # int
        CATEGORICAL_FEATURES.append(stats.datasets[0].features[_].path.step[0])
    elif stats.datasets[0].features[_].type == 1:  # float
        NUMERIC_FEATURES.append(stats.datasets[0].features[_].path.step[0])
    elif stats.datasets[0].features[_].type == 2:  # string
        CATEGORICAL_FEATURES.append(stats.datasets[0].features[_].path.step[0])

### 保留特征列信息

接下来，您需要保留数据集中特征列的信息。在此示例中，您将将这些用户定义的元数据作为 JSON 文件添加到与数据集关联的 Cloud Storage 存储桶中。

In [ ]:
import json

metadata = {
    "label_column": label_column,
    "numeric_features": NUMERIC_FEATURES,
    "categorical_features": CATEGORICAL_FEATURES,
}
with tf.io.gfile.GFile(
    "gs://" + dataset.labels["user_metadata"] + "/metadata.jsonl", "w"
) as f:
    json.dump(metadata, f)

保留数据集的统计信息

接下来，您将数据集的统计信息写入数据集的云存储桶，并保留统计文件的云存储位置。在这个例子中，您将其添加到存储在数据集云存储桶中的此数据集的用户定义元数据中。

In [ ]:
STATISTICS_SCHEMA = BUCKET_URI + "/statistics.jsonl"

tfdv.write_stats_text(stats, BUCKET_URI + "/statistics.jsonl")

with tf.io.gfile.GFile(
    "gs://" + dataset.labels["user_metadata"] + "/metadata.jsonl", "r"
) as f:
    metadata = json.load(f)

metadata["statistics"] = STATISTICS_SCHEMA
with tf.io.gfile.GFile(
    "gs://" + dataset.labels["user_metadata"] + "/metadata.jsonl", "w"
) as f:
    json.dump(metadata, f)

! gsutil cat $BUCKET_URI/metadata.jsonl

### 生成原始数据模式

使用 TensorFlow 数据验证（TFDV）包在数据集上生成数据模式。使用 `infer_schema()` 方法，并设置以下参数：

- `statistics`：TFDV 生成的统计信息。

In [ ]:
schema = tfdv.infer_schema(statistics=stats)
print(schema)

将数据集的模式保存到云存储

接下来，您将数据集的模式编写到数据集的云存储存储桶中。

In [ ]:
SCHEMA_LOCATION = BUCKET_URI + "/schema.txt"

# When running Apache Beam directly (file is directly accessed)
tfdv.write_schema_text(output_path=SCHEMA_LOCATION, schema=schema)
# When running with Dataflow (file is uploaded to worker pool)
tfdv.write_schema_text(output_path="schema.txt", schema=schema)

保留数据集的模式

接下来，您保留模式文件的云存储位置。在这个示例中，您将其添加到存储在数据集的云存储桶中的用户定义的元数据中。

In [ ]:
with tf.io.gfile.GFile(
    "gs://" + dataset.labels["user_metadata"] + "/metadata.jsonl", "r"
) as f:
    metadata = json.load(f)
metadata["schema"] = SCHEMA_LOCATION

with tf.io.gfile.GFile(
    "gs://" + dataset.labels["user_metadata"] + "/metadata.jsonl", "w"
) as f:
    json.dump(metadata, f)

! gsutil cat $BUCKET_URI/metadata.jsonl

生成特征规范，与TFRecords兼容，在具有TensorFlow Transform (TFT)包的数据集上。使用`schema_as_feature_spec()`方法，具有以下参数：

- `schema`：由TFDV生成的数据模式。

In [ ]:
feature_spec = tft.tf_metadata.schema_utils.schema_as_feature_spec(schema).feature_spec

print(feature_spec)

准备数据流作业的软件包要求。

在运行数据流作业之前，您需要为执行作业的工作池指定软件包要求。

In [ ]:
%%writefile setup.py
import setuptools

REQUIRED_PACKAGES = [
    "google-cloud-aiplatform",
    "tensorflow-transform==1.2.0",
    "tensorflow-data-validation==1.2.0",
]

setuptools.setup(
    name="executor",
    version="0.0.1",
    install_requires=REQUIRED_PACKAGES,
    packages=setuptools.find_packages(),
    include_package_data=True,
    package_data={"./": ["schema.txt"]}
)

创建预处理函数

接下来，您将创建一个特定于数据集的预处理函数。在这个例子中，您将预处理函数写入一个单独的Python模块，并添加一个__init__.py以使其看起来像一个包。为什么？当您在Dataflow中运行Apache Beam管道时，您的脚本会在一个或多个工作程序上运行。预处理函数在与管道不同的工作程序中运行，因此不包含管道的运行时，如全局变量的值。要解决这个问题，您需要将所有依赖项和值硬编码到预处理包中。

In [ ]:
! rm -rf src
! mkdir src
! touch src/__init__.py

with open("src/features.py", "w") as f:
    f.write("import tensorflow as tf\n")
    f.write("import tensorflow_transform as tft\n")

    f.write("def preprocessing_fn(inputs):\n")
    f.write("	outputs = {}\n")
    f.write("	for key in inputs.keys():\n")
    f.write(f"		if key in {NUMERIC_FEATURES}:\n")
    f.write("			outputs[key] = tft.scale_to_z_score(inputs[key])\n")
    f.write(f"		elif key in {CATEGORICAL_FEATURES}:\n")
    f.write("			outputs[key] = tft.compute_and_apply_vocabulary(\n")
    f.write("				inputs[key],\n")
    f.write("				num_oov_buckets=1,\n")
    f.write("				vocab_filename=key,\n")
    f.write("			)\n")
    f.write("		else:\n")
    f.write("			outputs[key] = inputs[key]\n")
    f.write("		outputs[key] = tf.squeeze(outputs[key], -1)\n")
    f.write("	return outputs\n")

### 使用Dataflow预处理数据

#### 数据预处理

接下来，您将使用Dataflow预处理数据。在这个例子中，您将查询BigQuery表，并将示例拆分成训练、验证和测试（评估）数据集，并对特征列进行预处理：

- `Numeric`：使用 `tft.scale_to_z_score` 重新缩放值。
- `Categorical`：使用 `tft.compute_and_apply_vocabulary` 将其编码为分类列。

除了经过预处理（转换）的数据之外，还会生成原始版本的测试数据，分别以tf.Example和JSONL格式存储。转换工件也会被保存，以供后续的服务功能将原始数据转换为经过转换的数据。

总而言之，生成的输出包括：

- 经过转换的训练数据
- 经过转换的验证数据
- 经过转换的测试数据
- 以JSONL格式存储的原始测试数据
- 以tf.Example格式存储的原始测试数据
- 转换函数工件

In [ ]:
import os

import tensorflow_transform.beam as tft_beam
from src import features

RUNNER = "DataflowRunner"  # DirectRunner for local running w/o Dataflow


def parse_bq_record(bq_record):
    """Parses a bq_record to a dictionary."""
    output = {}
    for key in bq_record:
        output[key] = [bq_record[key]]
    return output


def split_dataset(bq_row, num_partitions, ratio):
    """Returns a partition number for a given bq_row."""
    import json

    assert num_partitions == len(ratio)
    bucket = sum(map(ord, json.dumps(bq_row))) % sum(ratio)
    total = 0
    for i, part in enumerate(ratio):
        total += part
        if bucket < total:
            return i
    return len(ratio) - 1


def convert_to_jsonl(data, label=None):
    """Converts a parsed record to JSON"""
    import json

    if label:
        del data[label]
    return json.dumps(data)


def run_pipeline(args):
    """Runs a Beam pipeline to split the dataset"""

    pipeline_options = beam.pipeline.PipelineOptions(flags=[], **args)

    raw_data_query = args["raw_data_query"]
    label = args["label"]
    transformed_data_prefix = args["transformed_data_prefix"]
    transform_artifact_dir = args["transform_artifact_dir"]
    exported_jsonl_prefix = args["exported_jsonl_prefix"]
    exported_tfrec_prefix = args["exported_tfrec_prefix"]
    temp_location = args["temp_location"]
    project = args["project"]

    schema = tfdv.load_schema_text(SCHEMA_LOCATION)
    feature_spec = tft.tf_metadata.schema_utils.schema_as_feature_spec(
        schema
    ).feature_spec

    raw_metadata = tft.tf_metadata.dataset_metadata.DatasetMetadata(
        tft.tf_metadata.schema_utils.schema_from_feature_spec(feature_spec)
    )

    with beam.Pipeline(options=pipeline_options) as pipeline:
        with tft_beam.Context(temp_location):

            # Read raw BigQuery data.
            raw_train_data, raw_val_data, raw_test_data = (
                pipeline
                | "Read Raw Data"
                >> beam.io.ReadFromBigQuery(
                    query=raw_data_query,
                    project=project,
                    use_standard_sql=True,
                )
                | "Parse Data" >> beam.Map(parse_bq_record)
                | "Split" >> beam.Partition(split_dataset, 3, ratio=[8, 1, 1])
            )

            # Create a train_dataset from the data and schema.
            raw_train_dataset = (raw_train_data, raw_metadata)

            # Analyze and transform raw_train_dataset to produced transformed_train_dataset and transform_fn.
            transformed_train_dataset, transform_fn = (
                raw_train_dataset
                | "Analyze & Transform"
                >> tft_beam.AnalyzeAndTransformDataset(features.preprocessing_fn)
            )

            # Get data and schema separately from the transformed_dataset.
            transformed_train_data, transformed_metadata = transformed_train_dataset

            # write transformed train data.
            _ = (
                transformed_train_data
                | "Write Transformed Train Data"
                >> beam.io.tfrecordio.WriteToTFRecord(
                    file_path_prefix=os.path.join(
                        transformed_data_prefix, "train/data"
                    ),
                    file_name_suffix=".gz",
                    coder=tft.coders.ExampleProtoCoder(transformed_metadata.schema),
                )
            )

            # Create a val_dataset from the data and schema.
            raw_val_dataset = (raw_val_data, raw_metadata)

            # Transform raw_val_dataset to produced transformed_val_dataset using transform_fn.
            transformed_val_dataset = (
                raw_val_dataset,
                transform_fn,
            ) | "Transform Validation Data" >> tft_beam.TransformDataset()

            # Get data from the transformed_val_dataset.
            transformed_val_data, _ = transformed_val_dataset

            # write transformed val data.
            _ = (
                transformed_val_data
                | "Write Transformed Validation Data"
                >> beam.io.tfrecordio.WriteToTFRecord(
                    file_path_prefix=os.path.join(transformed_data_prefix, "val/data"),
                    file_name_suffix=".gz",
                    coder=tft.coders.ExampleProtoCoder(transformed_metadata.schema),
                )
            )

            # Create a test_dataset from the data and schema.
            raw_test_dataset = (raw_test_data, raw_metadata)

            # Transform raw_test_dataset to produced transformed_test_dataset using transform_fn.
            transformed_test_dataset = (
                raw_test_dataset,
                transform_fn,
            ) | "Transform Test Data" >> tft_beam.TransformDataset()

            # Get data from the transformed_test_dataset.
            transformed_test_data, _ = transformed_test_dataset

            # write transformed test data.
            _ = (
                transformed_test_data
                | "Write Transformed Test Data"
                >> beam.io.tfrecordio.WriteToTFRecord(
                    file_path_prefix=os.path.join(transformed_data_prefix, "test/data"),
                    file_name_suffix=".gz",
                    coder=tft.coders.ExampleProtoCoder(transformed_metadata.schema),
                )
            )

            # Write transform_fn.
            _ = transform_fn | "Write Transform Artifacts" >> tft_beam.WriteTransformFn(
                transform_artifact_dir
            )

            # Write raw test data to GCS as TF Records
            _ = (
                raw_test_data
                | "Write TF Test Data"
                >> beam.io.tfrecordio.WriteToTFRecord(
                    file_path_prefix=os.path.join(exported_tfrec_prefix, "data"),
                    file_name_suffix=".tfrecord",
                    coder=tft.coders.ExampleProtoCoder(raw_metadata.schema),
                )
            )

            # Convert raw test data to JSON (for batch prediction)
            json_test_data = (raw_test_data) | "Convert Batch Test Data" >> beam.Map(
                convert_to_jsonl, label=label
            )

            # Write raw test data to GCS as JSONL files.
            _ = json_test_data | "Write JSONL Test Data" >> beam.io.WriteToText(
                file_path_prefix=exported_jsonl_prefix, file_name_suffix=".jsonl"
            )


EXPORTED_JSONL_PREFIX = os.path.join(BUCKET_URI, "exported_data/jsonl")
EXPORTED_TFREC_PREFIX = os.path.join(BUCKET_URI, "exported_data/tfrec")
TRANSFORMED_DATA_PREFIX = os.path.join(BUCKET_URI, "transformed_data")
TRANSFORM_ARTIFACTS_DIR = os.path.join(BUCKET_URI, "transformed_artifacts")

QUERY_STRING = "SELECT * FROM {} LIMIT 300000".format(BQ_TABLE)
JOB_NAME = "chicago" + TIMESTAMP

args = {
    "runner": RUNNER,
    "raw_data_query": QUERY_STRING,
    "label": label_column,
    "transformed_data_prefix": TRANSFORMED_DATA_PREFIX,
    "transform_artifact_dir": TRANSFORM_ARTIFACTS_DIR,
    "exported_jsonl_prefix": EXPORTED_JSONL_PREFIX,
    "exported_tfrec_prefix": EXPORTED_TFREC_PREFIX,
    "temp_location": os.path.join(BUCKET_URI, "temp"),
    "project": PROJECT_ID,
    "region": REGION,
    "setup_file": "./setup.py",
}

print("Data preprocessing started...")
run_pipeline(args)
print("Data preprocessing completed.")

! gsutil ls $TRANSFORMED_DATA_PREFIX/train
! gsutil ls $TRANSFORMED_DATA_PREFIX/val
! gsutil ls $TRANSFORMED_DATA_PREFIX/test
! gsutil ls $TRANSFORM_ARTIFACTS_DIR
! gsutil ls {EXPORTED_JSONL_PREFIX}*
! gsutil ls $EXPORTED_TFREC_PREFIX

#### 保留转换后数据集的引用

接下来，您需要保留转换后的数据集与数据集的Cloud Storage位置的关联。在本例中，您将其添加到存储在数据集的Cloud Storage存储桶中的用户定义的元数据中。

在数据转换过程中，转换函数计算了每个特征的唯一出现次数。一些分类值（字符串，整数）可能具有很多唯一值。在这种情况下，最好通过将其从分类变为嵌入特征来降低它们的维度。

该代码使用了一个经验法则，即嵌入大小应该是唯一值的平方根。

In [ ]:
with tf.io.gfile.GFile(
    "gs://" + dataset.labels["user_metadata"] + "/metadata.jsonl", "r"
) as f:
    metadata = json.load(f)

tft_output = tft.TFTransformOutput(TRANSFORM_ARTIFACTS_DIR)

CATEGORICAL_FEATURES = []
EMBEDDING_FEATURES = []
categorical_features = metadata["categorical_features"]
for feature in categorical_features:
    unique = tft_output.vocabulary_size_by_name(feature)
    if unique > 10:
        EMBEDDING_FEATURES.append(feature)
        print("Convert to embedding", feature, unique)
    else:
        CATEGORICAL_FEATURES.append(feature)

metadata["categorical_features"] = CATEGORICAL_FEATURES
metadata["embedding_features"] = EMBEDDING_FEATURES

metadata["transformed_data_prefix"] = TRANSFORMED_DATA_PREFIX
metadata["transform_artifacts_dir"] = TRANSFORM_ARTIFACTS_DIR
metadata["exported_jsonl_prefix"] = EXPORTED_JSONL_PREFIX
metadata["exported_tfrec_prefix"] = EXPORTED_TFREC_PREFIX
with tf.io.gfile.GFile(
    "gs://" + dataset.labels["user_metadata"] + "/metadata.jsonl", "w"
) as f:
    json.dump(metadata, f)

! gsutil cat $BUCKET_URI/metadata.jsonl

清理

为了清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

*注意:* stage2/mlops_experimentation 依赖于由stage1笔记本创建的资源。

In [ ]:
delete_all = False

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    if "BUCKET_URI" in globals():
        ! gsutil rm -r $BUCKET_URI